In [2]:
import pandas as pd
import ibis
from ibis import selectors as s 
from ibis import _
ibis.options.interactive=True

data = pd.read_csv('2024-02-02_-_Worker_and_Temporary_Worker.csv')

In [13]:
t=ibis.memtable(data)
t_clean=(
    t
    .mutate(town=_["Town/City"].upper())
    .filter(_.Route == 'Skilled Worker')
)
t_town=(
    t_clean
    .group_by(_.town)
    .aggregate(n=_.count())
    .order_by(_.n.desc())
)

display(t_town.head(10))
t_town.filter(_.town=='EXETER')


┏━━━━━━━━━━━━┳━━━━━━━┓
┃ town       ┃ n     ┃
┡━━━━━━━━━━━━╇━━━━━━━┩
│ string     │ int64 │
├────────────┼───────┤
│ LONDON     │ 24901 │
│ BIRMINGHAM │  1962 │
│ MANCHESTER │  1814 │
│ BRISTOL    │   891 │
│ LEICESTER  │   878 │
│ EDINBURGH  │   748 │
│ LEEDS      │   736 │
│ GLASGOW    │   732 │
│ NOTTINGHAM │   700 │
│ READING    │   664 │
└────────────┴───────┘

┏━━━━━━━━┳━━━━━━━┓
┃ town   ┃ n     ┃
┡━━━━━━━━╇━━━━━━━┩
│ string │ int64 │
├────────┼───────┤
│ EXETER │   124 │
└────────┴───────┘

In [4]:
t_clean.filter(_.town=='EXETER')

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━┓
┃ Organisation Name                   ┃ Town/City ┃ County ┃ Type & Rating     ┃ Route          ┃ town   ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━┩
│ string                              │ string    │ string │ string            │ string         │ string │
├─────────────────────────────────────┼───────────┼────────┼───────────────────┼────────────────┼────────┤
│ 3DFORUS Limited                     │ Exeter    │ NULL   │ Worker (A rating) │ Skilled Worker │ EXETER │
│ A J Sands Ltd T/A Sands Consultants │ Exeter    │ Devon  │ Worker (A rating) │ Skilled Worker │ EXETER │
│ A&S 2014 Ltd                        │ Exeter    │ NULL   │ Worker (A rating) │ Skilled Worker │ EXETER │
│ A2Z SPICE LTD                       │ exeter    │ NULL   │ Worker (A rating) │ Skilled Worker │ EXETER │
│ ABBEY 24X7 LIMITED                  │ EXETER    │ NULL   │ Worker (A rating) │ Skilled Worker │ EXETER │
│ Alexander House                     │ Exeter    │ NULL   │ Worker (A rating) │ Skilled Worker │ EXETER │
│ Amelia House Residential House      │ Exeter    │ Devon  │ Worker (A rating) │ Skilled Worker │ EXETER │
│ AnTech Ltd                          │ Exeter    │ Devon  │ Worker (A rating) │ Skilled Worker │ EXETER │
│ Appen (Europe) LTD                  │ Exeter    │ Devon  │ Worker (A rating) │ Skilled Worker │ EXETER │
│ Artigiano Espresso Bars limited     │ Exeter    │ NULL   │ Worker (A rating) │ Skilled Worker │ EXETER │
│ …                                   │ …         │ …      │ …                 │ …              │ …      │
└─────────────────────────────────────┴───────────┴────────┴───────────────────┴────────────────┴────────┘

### Check if Company has Sponsor

In [5]:
t.filter(_['Organisation Name'].contains('Vitrifi'))

┏━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━┳━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┓
┃ Organisation Name ┃ Town/City ┃ County ┃ Type & Rating     ┃ Route          ┃
┡━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━╇━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━┩
│ string            │ string    │ string │ string            │ string         │
├───────────────────┼───────────┼────────┼───────────────────┼────────────────┤
│ Vitrifi Limited   │ London    │ NULL   │ Worker (A rating) │ Skilled Worker │
└───────────────────┴───────────┴────────┴───────────────────┴────────────────┘

### Get a list of Company House Companies

In [6]:
import requests
from requests.auth import HTTPBasicAuth as Auth

# stream_key='ce33ea20-3dd9-4cbe-8a57-1ad0396d7c78'
rest_key = 'a66a1cd0-cf10-4c36-9759-6a4d0b3b8053'
# steam_url='https://stream.companieshouse.gov.uk/companies'
rest_url='https://api.company-information.service.gov.uk/advanced-search/companies'
r = requests.get(
    rest_url,
    auth=(rest_key,'')
    )
assert r.status_code==200
cps = r.json().get('items')

In [7]:
rest_url='https://api.company-information.service.gov.uk/search'
r = requests.get(
    rest_url,
    auth=(rest_key,''),
    params={'q':'Fibre'}
    )
assert r.status_code==200
cps = r.json().get('items')

In [8]:
cps

[{'address_snippet': 'Victoria House Suite 2, Victoria House, South Street, Farnham, Surrey, England, GU9 7QU',
  'description_identifier': ['incorporated-on'],
  'date_of_creation': '2018-06-25',
  'company_status': 'active',
  'company_type': 'ltd',
  'company_number': '11430513',
  'address': {'address_line_1': 'South Street',
   'locality': 'Farnham',
   'region': 'Surrey',
   'premises': 'Victoria House Suite 2, Victoria House',
   'postal_code': 'GU9 7QU',
   'country': 'England'},
  'description': '11430513 - Incorporated on 25 June 2018',
  'kind': 'searchresults#company',
  'matches': {'title': [1, 5], 'snippet': []},
  'snippet': '',
  'title': 'FIBRE LIMITED',
  'links': {'self': '/company/11430513'}},
 {'links': {'self': '/company/10497150'},
  'title': 'FIBRE LTD',
  'snippet': '',
  'matches': {'title': [1, 5], 'snippet': []},
  'kind': 'searchresults#company',
  'description': '10497150 - Dissolved on  1 May 2018',
  'address': {'premises': '212 ',
   'region': 'Merseysi